$$
\newcommand{\Lang}{\mathcal{L}}
\newcommand{\LangInt}{\Lang_\mathsf{int}}
\newcommand{\LangVar}{\Lang_\mathsf{var}}
\newcommand{\LangXInt}{\mathrm{x86}_\mathsf{int}}
\newcommand{\LangXVar}{\mathrm{x86}_\mathsf{var}}
\newcommand{\LangX}{\mathrm{x86}}
\newcommand{\INPUT}{\texttt{input}\_\texttt{int}}
\newcommand{\PRINT}[1]{\texttt{print}( #1 )}
\newcommand{\EXP}{\texttt{Expr}}
\newcommand{\CALL}[2]{\EXP(\texttt{Call}(\texttt{Name}(\texttt{'}#1\texttt{'}), [ #2 ])}
\newcommand{\CPRINT}[1]{\CALL{\texttt{print}}{#1}}
\newcommand{\CINPUT}{\CALL{\INPUT}{}}
\newcommand{\MODULE}[1]{\texttt{Module}( #1 )}
\newcommand{\CONST}{\texttt{Constant}(int)}
\newcommand{\NEG}[1]{\texttt{UnaryOp}(\texttt{USub}(), #1)}
\newcommand{\UOP}[1]{\texttt{UnaryOp}(\texttt{unaryop}, #1)}
\newcommand{\ADD}[2]{\texttt{BinOp}(#1, \texttt{Add}(), #2)}
\newcommand{\SUB}[2]{\texttt{BinOp}(#1, \texttt{Sub}(), #2)}
\newcommand{\BOP}[2]{\texttt{BinOp}(#1, \texttt{binaryop}, #2)}
\newcommand{\VAR}[1]{\texttt{Name}( #1 )}
\newcommand{\ASSIGN}[2]{\texttt{Assign}([\texttt{Name}( #1 )], #2)}
\newcommand{\RSP}{\texttt{'rsp'}}
\newcommand{\RBP}{\texttt{'rbp'}}
\newcommand{\RAX}{\texttt{'rax'}}
\newcommand{\RBX}{\texttt{'rbx'}}
\newcommand{\RCX}{\texttt{'rcx'}}
\newcommand{\RDX}{\texttt{'rdx'}}
\newcommand{\RSI}{\texttt{'rsi'}}
\newcommand{\RDI}{\texttt{'rdi'}}
\newcommand{\RN}[1]{\texttt{'r#1'}}
\newcommand{\ADDQ}[2]{\texttt{addq}\ #1, #2}
\newcommand{\SUBQ}[2]{\texttt{subq}\ #1, #2}
\newcommand{\NEGQ}[1]{\texttt{negq}\ #1}
\newcommand{\MOVQ}[2]{\texttt{movq}\ #1, #2}
\newcommand{\CALLQ}{\texttt{Callq}(label,int)}
\newcommand{\PUSHQ}[1]{\texttt{pushq}\ #1}
\newcommand{\POPQ}[1]{\texttt{popq}\ #1}
\newcommand{\RETQ}{\texttt{Retq}()}
\newcommand{\JUMP}{\texttt{Jump}(label)}
\newcommand{\INSTRun}[1]{\texttt{Instr}(\texttt{'#1'},[arg])}
\newcommand{\INSTRbin}[1]{\texttt{Instr}(\texttt{'#1'},[arg,arg])}
$$

# Lektion 2: Zwischencodeoptimierung - Register Allocation

====================================================================================================================

#### Erinnerung

- *bisher:* Programmvariablen wurden im `assign_homes`-Pass im Speicher verwaltet
    - so nicht in der Praxis zu gebrauchen 
- **Registerzugriffe sind um vielfaches effizienter** als Speicherzugriffe
- *aber:* nur 16 Register verfügbar

#### Ziel

**Programmvariablen (weitestgehend) in Registern speichern**

- *gesucht:* eine "injektive Abbildung" von Variablen auf Register (**Welche Bedingungen sollte diese Zuordnung erfüllen?**)
- *klar:* i.A. mehr Variablen als Register
    - *Was passiert in diesem Fall?*
- daher ist ein "cleveres" Vorgehen nötig
    - dafür benötigte Information: *Wann im Programmablauf wird welche Variable benutzt?*

====================================================================================================================

## Live-Variable (Liveness Analysis)

im Folgenden bezeichnen wir Variablen, Stackzugriffe und Register als **locations**

#### Definitionen:
Sei $I_1,...,I_n$ eine Folge von Instruktionen aus $\LangXVar$, mit

- $\color{green}{W(k)}$ bezeichnen wir die Menge der locations die durch $I_k$ *beschrieben* werden
- $\color{green}{R(k)}$ bezeichnen wir die Menge der locations die durch $I_k$ *gelesen* werden
- $\color{green}{L_\mathsf{before}(k)}$, $\color{green}{L_\mathsf{after}(k)}$ bezeichnen wir die Menge der locations die *vor* und *nach* $I_k$ live sind, diese sind dabei durch folgende Gleichungen definiert:

$$\begin{align}
L_\mathsf{after}(n) & = \emptyset \\
L_\mathsf{before}(k) & = (L_\mathsf{after}(k)\setminus W(k))\cup R(k) \\
L_\mathsf{after}(k) & = L_\mathsf{before}(k+1) 
\end{align}
$$

====================================================================================================================

#### Beispiel

```
 1 movq $1, v
 2 movq $42, w
 3 movq v, x
 4 addq $7, x
 5 movq x, y
 6 movq x, z
 7 addq w, z
 8 movq y, tmp_0
 9 negq tmp_0
10 movq z, tmp_1
11 addq tmp_0, tmp_1
12 movq tmp_1, %rdi
13 callq print_int
```

$$\begin{align}
L_\mathsf{after}(n) & = \emptyset \\
L_\mathsf{before}(k) & = (L_\mathsf{after}(k)\setminus W(k))\cup R(k) \\
L_\mathsf{after}(k) & = L_\mathsf{before}(k+1) 
\end{align}
$$

====================================================================================================================

## Register Allocation mit Hilfe des Färbbarkeitsproblems für Graphen


#### Idee

- *locations*, die am selben Programmpunkt *live* sind, dürfen nicht dem selben Register zugeordnet werden
    - diese Informationen lässt sich in einer Datenstruktur abbilden

====================================================================================================================

### Register Interference Graph (RIG)

#### Definition

Sei $I_1,...,I_n$ eine Folge von Instruktionen aus $\LangXVar$. Ein RIG ist ein ungerichteter Graph $\mathcal{G}=(V,E)$
- dessen Knoten $V$ die locations sind und 
- die Kanten wie folgt für alle $i\leq n$ definiert sind
    1. Wenn $I_i=$ `movq s,d`, dann füge für jedes $v\in L_\mathsf{after}(i)$ mit $v\neq s$ **und** $v\neq d$ eine Kante $(d,v)$ hinzu
    
    2. Wenn $I_i$ ist keine `movq`-Instruktion, dann füge für jedes $v\in W(i)$ und jedes $d\in L_\mathsf{after}(i)$ eine Kante $(v,d)$ hinzu, falls $v\neq d$


**Beispiel:** Tafel!

====================================================================================================================

#### Wie kann man den RIG nutzen?

Einfach indem **je 2 Variablen $u,v$ die Nachbarn sind**, also $(u,v)\in E$ gilt, **unterschiedliche Register** zugewiesen werden!

**Hmm...?**

Das Problem ist doch bekannt!

*Definition:* **$k$-färbbarkeits-Problem für Graphen**
- geg.: ungerichteter Graph und $k$ Farben
- Frage: Gibt es eine Zuordnung von Knoten zu Farben, so dass je 2 benachbarte Knoten nicht die gleiche Farbe haben?

Färbbarkeit (für $k\geq 3$) ist **NP-vollständig**! -> keine polynomiellen Algorithmen (bekannt)

#### Was kann man tun?

Einen heuristischen Ansatz benutzen, der sich als ausreichend performant in der Praxis herausgestellt hat.

====================================================================================================================

### Chaitins Algorithmus

- letztlich ein **greedy**-Algorithmus zum "Lösen" des $k$-Färbbarkeitsproblems
- Gregory Chaitin schlug erstmals vor, $k$-Färbbarkeit zum Lösen des register-allocation-Problems zu nutzen (1981)
- vorher keine guten Algo. zum Lösen dieses Problems
- die hier präsentierten Ideen sind z.B. im gcc im Einsatz
- *hier:* zunächst erstmal nur in vereinfachter Form
    - später werden wir noch Heuristiken zur Verbesserung des Algos. kennenlernen

#### Idee

- Annahme: Graph $\mathcal{G}$ soll $k$-gefärbt werden
- wähle eine Knoten mit weniger als $k$ Nachbarn und entferne diesen

- nun gilt: Falls der resultierende Graph $k$-färbbar ist, dann auch $\mathcal{G}$ 

**Warum**?    

#### Algo

- Input: $k$ Register (das sind die Farben), RIG
- wenn es noch mehr als einen Knoten gibt
    - wähle Knoten mit weniger als $k$ Nachbarn und lege diesen auf einen Stack
    - mache rekursiv weiter
- färbe die Knoten in der Stack-Reihenfolge 


====================================================================================================================

#### Probleme

- obiger Algo. ist letztlich ein polynomielles Verfahren, um einen Graphen zu färben, aber eine **Heuristik**
    - beruht auf der Annahme, dass sich immer ein Knoten mit weniger als $k$ Nachbarn finden lässt 
    - sogenannte *optimistische Färbung*
- *Was kann man tun, falls **kein** Knoten dieser Art gefunden wird oder der Graph überhaupt nicht färbbar ist?* ->

**Spilling**

- falls kein Knoten mit weniger als $k$ Nachbarn gefunden werden kann
    - markiere einen als `possibly_spilled` und lege diesen auf den Stack
    - verfahre weiter mit obigem Algo.
    

- *Beobachtung:* der ausgewählte Knoten kann im Färbungsprozess möglicherweise doch noch gefärbt werden, ansonsten wird ihm eine Speicheradresse zugewiesen

====================================================================================================================

#### Verbesserungen

**Auswahl der zu *spillenden* Variable** -- Heuristiken:

- Variablen mit den meisten Nachbarn zuerst
- Variblen die wenig im Code benutzt werden
- keine Variablen in Schleifen

**Aufspalten von Live-Ranges**

- **Variablen** die über "weite Strecken" inaktiv sind bei späterer wiederverwendung **umbenennen**
- dann Graph färben

**Coalescing**

- *Beobachtung:* Falls `x,y` dem selben Register zugewiesen wurden, dann kann eine Instruktion `movq x,y` aus dem Code entfernt werden.
- *Vorgehen:* 
    - zunächst speichern welche Variablen durch `movq`-Instruktionen verbunden sind (z.B. in Graph)
    - beim Färben versuchen, Knoten die `movq`-verbunden sind, anderen Knoten vorzuziehen und diesen dann das gleiche Register zuweisen

====================================================================================================================